# CNN for Image Classification

In this lab, you are given a dataset containing 6,000 pictures of cats and dogs (3,000 cats, 3,000 dogs) and asked to train a classifier built upon Convolutional Neural Networks (ConvNets) to classify images as "dogs" or "cats".

In [14]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import random

### Exercise 1 
Split the dataset by selecting 4,800 pictures for training, 600 for validation, and 600 for testing.

In [87]:
img_width = 50
img_height = 50
num_classes = 2
DATA_DIR = './data/'
image_filenames = [DATA_DIR+i for i in os.listdir(DATA_DIR)] # use this for full dataset
# Split the data in three sets, 80% for training, 10% for validation and 10% for testing
# make sure that the image filenames have a fixed order before shuffling
# Add your code here

random.shuffle(image_filenames)
x, y=[],[]

#ppedn each image file ot x and each label to y
for img in image_filenames:
    #reading image into array and then resize it to 50x50 #OPT: interpolation=INTER_AREA
    x.append(cv2.resize(cv2.imread(img),(img_height, img_width)))
    #use file name to detect label and appaend it to y
    if 'cat' in img:
        y.append(0)
    else:
        y.append(1)

#split the dats into trait, test and validation
x_train,x_test,x_valid=x[0:4800],x[4800:5400],x[5400:6000]
y_train,y_test,y_valid=y[0:4800],y[4800:5400],y[5400:6000]


### Exercise 2
Train a Convolutional Neural Network (ConvNet) on the training set. The general structure of the ConvNet will be a stack of alternated Conv2D (with relu activation) and MaxPooling2D layers. A Conv2D layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. A MaxPooling2D layer is used to downscale input in both the vertical and horizontal dimensions.

In [127]:
# Add your code here
batch=16
epochs=10

model =Sequential()
model.add(Conv2D(15,(4,4), input_shape=(img_height,img_width,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Flatten())
#add a feedforeward layer with 16 neurons 
model.add(Dense(25))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Gen=ImageDataGenerator(rescale=1./255)
trainGen=Gen.flow(np.array(x_train), y_train, batch_size=batch, shuffle=False)
validGen=Gen.flow(np.array(x_valid), y_valid, batch_size=batch, shuffle=False)

history = model.fit_generator(
    trainGen, 
    steps_per_epoch=4800//batch,
    epochs=epochs,
    validation_data=validGen,
    validation_steps=600//batch
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_69 (Conv2D)           (None, 47, 47, 8)         392       
_________________________________________________________________
activation_169 (Activation)  (None, 47, 47, 8)         0         
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 15, 15, 8)         0         
_________________________________________________________________
flatten_62 (Flatten)         (None, 1800)              0         
_________________________________________________________________
dense_103 (Dense)            (None, 24)                43224     
_________________________________________________________________
activation_170 (Activation)  (None, 24)                0         
_________________________________________________________________
dense_104 (Dense)            (None, 1)                 25        
__________

### Exercise 3

Output the training/validation loss and accuracy curves. Also print the classification results (e.g., classification accuracy, confusion matrix, precision-recall curves and/or ROC curves) on the test set.

In [124]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_curve, roc_curve

#saving the prediction probabilities in pred_train_y and pred_valid_Y
pred_train_y=model.predict(np.array(x_train), batch_size=batch)
pred_valid_y=model.predict(np.array(x_valid), batch_size=batch)

#outputing the classification report, accuracy and the confusion matrix of the training dataset
print(classification_report(y_train,pred_train_y.round()))
print(confusion_matrix(y_train,pred_train_y.round()))
print(accuracy_score(y_train,pred_train_y.round()))

#outputing the classification report, accuracy and the confusion matrix of the validation dataset
print(classification_report(y_valid,pred_valid_y.round()))
print(confusion_matrix(y_valid,pred_valid_y.round()))
print(accuracy_score(y_valid,pred_valid_y.round()))

# Add your code here

              precision    recall  f1-score   support

           0       0.87      0.86      0.87      2380
           1       0.87      0.87      0.87      2420

   micro avg       0.87      0.87      0.87      4800
   macro avg       0.87      0.87      0.87      4800
weighted avg       0.87      0.87      0.87      4800

[[2057  323]
 [ 310 2110]]
0.868125
              precision    recall  f1-score   support

           0       0.72      0.65      0.68       310
           1       0.66      0.72      0.69       290

   micro avg       0.69      0.69      0.69       600
   macro avg       0.69      0.69      0.69       600
weighted avg       0.69      0.69      0.69       600

[[202 108]
 [ 80 210]]
0.6866666666666666


### Exercise 4

Explore different network architectures (e.g., stacking 4 Conv2D+MaxPooling2D layers) and various ways in tuning the model parameters to see if you can improve the model performance on the validation set.

In [183]:
# Add your code here
batch=16
epochs=10

model = Sequential()
model.add(Conv2D(16,(3,3), input_shape=(img_height,img_width,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(24, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(24, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#add a feedforeward layer with 16 neurons 
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Gen=ImageDataGenerator(rescale=1./255)
trainGen=Gen.flow(np.array(x_train), y_train, batch_size=batch, shuffle=False)
validGen=Gen.flow(np.array(x_valid), y_valid, batch_size=batch, shuffle=False)

history = model.fit_generator(
    trainGen, 
    steps_per_epoch=4800//batch,
    epochs=epochs,
    validation_data=validGen,
    validation_steps=600//batch
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_198 (Conv2D)          (None, 48, 48, 16)        448       
_________________________________________________________________
activation_320 (Activation)  (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_179 (MaxPoolin (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_199 (Conv2D)          (None, 22, 22, 24)        3480      
_________________________________________________________________
activation_321 (Activation)  (None, 22, 22, 24)        0         
_________________________________________________________________
max_pooling2d_180 (MaxPoolin (None, 11, 11, 24)        0         
_________________________________________________________________
conv2d_200 (Conv2D)          (None, 9, 9, 24)          5208      
__________

### Exercise 5

Apply the trained model on the testing set and output the classification results.

In [185]:
# Add your code here

pred_test_y=model.predict(np.array(x_test), batch_size=batch)
#outputing the classification report, accuracy and the confusion matrix of the validation dataset
print(classification_report(y_test,pred_test_y.round()))
print(confusion_matrix(y_test,pred_test_y.round()))
print(accuracy_score(y_test,pred_test_y.round()))


              precision    recall  f1-score   support

           0       0.81      0.61      0.70       310
           1       0.67      0.85      0.75       290

   micro avg       0.72      0.72      0.73       600
   macro avg       0.74      0.73      0.72       600
weighted avg       0.74      0.72      0.72       600

[[189 121]
 [ 44 246]]
0.725


### Exercise 6 

Plot the saliency map of original image to see which part is important for making classification decisions. You can refer to the following blog article on how to generate visualisation results of the filters in the ConvNets.
https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html

In [ ]:
# Add your code here